In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
from custom_dataset import catDogDataset

In [2]:
#Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')              

In [3]:
# Hyperparameters
in_channel = 3
num_classes = 10
learning_rate = 1e-3 
batch_size = 1024
num_epochs = 5

In [4]:
model = torchvision.models.googlenet(pretrained = True)
# print(model)
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [5]:
#Define train and test loader
dataset = catDogDataset(csv_file = 'output.csv', root_dir = 'train_test_dataset',
                          transform = transforms.ToTensor())

In [6]:
train_set,test_set = torch.utils.data.random_split(dataset,[20000,5000])

train_loader = DataLoader(dataset = train_set, batch_size = batch_size , shuffle =True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size , shuffle =True)

In [7]:
#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [9]:
for epoch in range(num_epochs):
    losses = []
    
    for batch_idx,(data,targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent or adam step
        optimizer.step()

RuntimeError: stack expects each tensor to be equal size, but got [3, 351, 300] at entry 0 and [3, 443, 500] at entry 1